In [3]:
import tensorflow as tf
import tf2onnx

2024-11-14 17:52:47.191620: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 17:52:47.689980: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 17:52:47.692194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 17:52:49.288001: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load Model

In [7]:
model_path = 'RoadSegmentation'
model_ext = '.keras'

In [8]:
model = tf.keras.models.load_model(model_path+model_ext)

2024-11-14 17:53:16.319474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-14 17:53:16.322473: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Convert to Onnx

In [10]:
spec = (tf.TensorSpec((None, 128, 128, 3), tf.float32),)
model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, input_signature=spec)

2024-11-14 17:53:37.648302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-14 17:53:37.648874: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-11-14 17:53:37.649082: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-11-14 17:53:37.649622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-14 17:53:37.650148: W tensorflow/core/common_runtime/gpu/gpu_d

In [11]:
with open(model_path+".onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

# Convert to TFLite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpso97tyi_/assets


INFO:tensorflow:Assets written to: /tmp/tmpso97tyi_/assets
2024-11-14 17:53:52.789929: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-11-14 17:53:52.789961: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-11-14 17:53:52.791006: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpso97tyi_
2024-11-14 17:53:52.795268: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-11-14 17:53:52.795308: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpso97tyi_
2024-11-14 17:53:52.805979: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-11-14 17:53:52.809485: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-11-14 17:53:53.325467: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

In [13]:
with open(model_path+".tflite", 'wb') as f:
    f.write(tflite_model)